In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import FastText
import keras
import keras
from copy import deepcopy
import keras_contrib
from sklearn.metrics import f1_score,accuracy_score
from keras.models import load_model
from gensim.models import Word2Vec
import itertools
from more_itertools import locate
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight


Using TensorFlow backend.


In [2]:
with open('train_paras.txt','r') as f:
    data=f.readlines()
    
for i in range(0,len(data)):
    data[i]=data[i][2:].split("', '")
    data[i][-1]=data[i][-1].replace("']\n",'')

In [1]:
import pickle
with open("tags.pkl", "rb") as fp:
    tags = pickle.load(fp)

In [4]:
count=0
for i in range(0,len(data)):
    if i%100000==0 and i!=0:
        print(count)
    count+=len(data[i])


13647276
27286349
40969093
54709715
68387947
82038839
95745378


In [6]:
print(count)

107626405


In [7]:
start_index = 95745378
end_index = 107626405

In [8]:
data = data[700000:]

In [9]:
with open('one_hot_target.txt','r') as f:
    full_target=f.readlines()

In [10]:
full_target = full_target[start_index:end_index]

In [11]:
target=[]
i=0
k=0
while(i<end_index - start_index):
    new_sen=[]
    for j in range(0,len(data[k])):
        x=full_target[i+j].replace("[",'')
        x=x.replace("]\n",'')
        x=x.split(' ')
        x=[float(num) for num in x if num not in ""]
        new_sen.append(x)
    i+=len(data[k])
    k+=1
    target.append(np.array(new_sen))

In [12]:
del full_target
import gc
gc.collect()

0

In [15]:
new_target = []
for i in range(len(target)):
    temp = []
    for j in range(len(target[i])):
        temp.append(target[i][j][np.array([0,1,3,4])].tolist())
    new_target.append(temp)

In [16]:
del target
gc.collect()

0

In [17]:
fasttext_model = FastText.load('fasttext_big_new_test.model')
w2v_model=Word2Vec.load('word2vec_big_new_test.model')

In [18]:
gc.collect()

6

In [19]:
def transformation(x):
    for i in range(len(x)):
        x[i]=x[i][0:300]
        leng=300-len(x[i])
        x[i]=x[i]+leng*['pad']
        
    return x

In [20]:
def embed_fasttext(x):
    for i in range(len(x)):
        for j in range(300):
                try:
                    w2v = w2v_model.wv[x[i][j]]
                except:
                    w2v = np.zeros([150])
                try:
                    ft = fasttext_model.wv[x[i][j]]
                except:
                    ft = np.zeros([150])

                x[i][j] = np.concatenate([w2v,ft])

    
    return np.array(x)

In [21]:
input_text = keras.layers.Input(shape=(300,300))
lstm_1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(units=128, return_sequences=True))(input_text)
drop = keras.layers.Dropout(0.3)(lstm_1)
dense = keras.layers.TimeDistributed(keras.layers.Dense(70, activation="relu"))(drop)
out=keras.layers.Dense(4,activation='softmax')(dense)

model = keras.Model(inputs = input_text,outputs=out)

In [22]:
model.compile(optimizer="adam", loss="categorical_crossentropy")

In [23]:
model = keras.models.load_model('final_single_tags_model.h5')

In [24]:
def elmoembedding(x):
    x = transformation(x)
    y = embed_fasttext(deepcopy(x))
    return y

In [25]:
def target_processing(x):
    for i in range(len(x)):
        x[i] = x[i][0:300]
        for j in range(0,(300 - len(x[i]))):
            x[i] = np.vstack([x[i],np.array([1,0,0,0])])
    return np.array(x)

In [26]:
def generator(batch_size,from_list_x,from_list_y):

    assert len(from_list_x) == len(from_list_y)
    total_size = len(from_list_x)

    while True:

        for i in range(0,total_size,batch_size):
            yield np.array(elmoembedding(deepcopy(from_list_x[i:i+batch_size]))),target_processing((deepcopy(from_list_y[i:i+batch_size])))

In [27]:
model.fit_generator(generator=generator(64,data,new_target),epochs=4,steps_per_epoch=len(data)/64)

Epoch 1/4
1360/1359 [==============================] - 514s 378ms/step - loss: 0.0435
Epoch 2/4
1360/1359 [==============================] - 661s 486ms/step - loss: 0.0421
Epoch 3/4
1360/1359 [==============================] - 499s 367ms/step - loss: 0.0412
Epoch 4/4
1360/1359 [==============================] - 695s 511ms/step - loss: 0.0405


In [28]:
model.save('final_single_tags_model.h5')

In [39]:
merged = list(itertools.chain.from_iterable(tags))

dictionary_tags={}
for i in range(0,len(merged)):
    try:
        dictionary_tags[merged[i]]+=1
    except:
        dictionary_tags[merged[i]]=1
        
sorted_by_value = sorted(dictionary_tags.items(), key=lambda kv: kv[1],reverse=True)

set_of_all_tags=[]
for i in range(0,len(sorted_by_value)):
    if sorted_by_value[i][0] not in set_of_all_tags:
        set_of_all_tags.append(sorted_by_value[i][0])

In [41]:
def look_up_two(word,para):
    for i in range(0,len(set_of_all_tags)):
        x=set_of_all_tags[i].split('-')
        
        if len(x)==2 and word==x[0] and x[1] in para:
            return [set_of_all_tags[i]]
        if len(x)==2 and word==x[1] and x[0] in para:
            return [set_of_all_tags[i]]
    return []

In [42]:
def look_up_three(word,para):
    for i in range(0,len(set_of_all_tags)):
        x=set_of_all_tags[i].split('-')
        
        if len(x)==3 and word==x[0] and x[1] in para and x[2] in para:
            return [set_of_all_tags[i]]
        if len(x)==3 and word==x[1] and x[0] in para and x[2] in para:
            return [set_of_all_tags[i]]
        if len(x)==3 and word==x[2] and x[0] in para and x[1] in para:
            return [set_of_all_tags[i]]
    return []

In [67]:
def predict(pred_data):
    
    dat = elmoembedding(deepcopy(pred_data))
    result = model.predict(dat)
    
    new_res=[]
    for i in range(0,len(result)):
        pred_data[i]=np.array(pred_data[i])
        res=[]
        for j in range(0,len(result[i])):
            res.append(np.argmax(result[i][j]))
        new_res.append(res) 
        
    del result
    
    main_tags = []
    
    for i in range(0,len(new_res)):
        main_tags.append(pred_data[i][np.where(np.array(new_res[i])==1)[0]])
        
    
    two_tags_all=[]
    for i in range(0,len(new_res)):
        two_tags_words=pred_data[i][np.where(np.array(new_res[i])==2)[0]]
        two_tags_temp=[]
        for j in range(0,len(two_tags_words)):
            two_tags_temp+=look_up_two(two_tags_words[j],pred_data[i])
        two_tags_all.append(two_tags_temp)
            
        
    three_tags_all=[]
    for i in range(0,len(new_res)):
        three_tags_words=pred_data[i][np.where(np.array(new_res[i])==3)[0]]
        three_tags_temp=[]
        for j in range(0,len(three_tags_words)):
            three_tags_temp+=look_up_three(three_tags_words[j],pred_data[i])
        three_tags_all.append(three_tags_temp)
    
    full_tags=[]
    for i in range(0,len(pred_data)):        
        full = list(main_tags[i]) +two_tags_all[i]+three_tags_all[i] 
        full_tags.append(list(set(full)))
    
    
    return full_tags

In [52]:
def f1score(pred,actual):
    inter=0
    for i in range(len(pred)):
        inter=len(list(set(actual).intersection(pred)))
    try:
        precision=inter/len(pred)
    except ZeroDivisionError:
        precision=0
    recall=inter/len(actual)
    
    try:
        to_return=((2*recall*precision)/(recall+precision))
    except ZeroDivisionError:
        to_return=0
    return to_return

In [61]:
def call_predictions():
    i=0
    predicted_tags=[]
    while(i<len(x_test)):
        pred=predict(x_test[i:i+64])
        for j in range(0,len(pred)):
            if 'angular' in pred[j]:
                ind=pred[j].index('angular')
                pred[ind]=['angularjs']
            if 'ruby' in pred[j]:
                ind=pred[j].index('ruby')
                pred[ind]=['ruby-on-rails']
            if 'node' in pred[j]:
                ind=pred[j].index('node')
                pred[ind]=['node.js']
            if 'react' in pred[j]:
                ind=pred[j].index('react')
                pred[ind]=['reactjs']
            if 'psql' in pred[j]:
                ind=pred[j].index('psql')
                pred[ind]=['postgresql']
            if 'python3' in pred[j]:
                ind=pred[j].index('python3')
                pred[ind]=['python-3.x']
            
            
        i+=64
        predicted_tags+=pred
    
    predicted_tags = codes_func(predicted_tags)
    
    for i in range(len(predicted_tags)):
        predicted_tags[i] = list(set(predicted_tags[i]))
        predicted_tags[i] = [i for i in predicted_tags[i] if i !='angular']
        predicted_tags[i] = [i for i in predicted_tags[i] if i !='ruby']
        predicted_tags[i] = [i for i in predicted_tags[i] if i !='node']
        predicted_tags[i] = [i for i in predicted_tags[i] if i !='react']
        predicted_tags[i] = [i for i in predicted_tags[i] if i !='psql']
        
    avg_f1=0
    for i in range(0,len(x_test)):
        avg_f1+=f1score(predicted_tags[i],tags_test[i])
    return (avg_f1/len(x_test)),predicted_tags[0:32]

In [46]:
codes_tag=[]
for i in range(0,len(tags_test)):
    try:
        codes_tag.append(tags_test[i][0])
    except IndexError:
        pass

In [47]:
with open("vectorizer_1.pkl", "rb") as fp:
    vectorizer_1 = pickle.load(fp)
with open("vectorizer_2.pkl", "rb") as fp:
    vectorizer_2 = pickle.load(fp)
with open("model_codes_1.pkl", "rb") as fp:
    model_1 = pickle.load(fp)
with open("model_codes_2.pkl", "rb") as fp:
    model_2 = pickle.load(fp)

In [ ]:
model_1.predict_proba(vectorizer_2.transform([' '.join(x_tes)]))

In [48]:
def codes_func(predicted_tags):
    for i in range(0,len(x_test)):
        prob1=model_1.predict_proba(vectorizer_1.transform([' '.join(x_test[i])]))
        prob2=model_2.predict_proba(vectorizer_2.transform([' '.join(x_test[i])]))
        ind1=np.argmax(prob1)
        ind2=np.argmax(prob2)
        if prob1[0][ind1]>0.40:
            predicted_tags[i].append((model_1.predict(vectorizer_1.transform([' '.join(x_test[i])]))[0]))
        elif prob2[0][ind2]>0.60:
            predicted_tags[i].append((model_2.predict(vectorizer_2.transform([' '.join(x_test[i])]))[0]))
        predicted_tags[i] = list(set(predicted_tags[i]))
    return predicted_tags

In [2]:
tags[0:50]

[['flash', 'http', 'websocket', 'communication', 'wireshark'],
 ['sql', 'postgresql', 'offset'],
 ['php', 'permissions', 'binary', 'bit-manipulation'],
 ['reactjs'],
 ['javascript', 'node.js', 'websocket', 'rpc', 'promise'],
 ['android'],
 ['java'],
 ['c++', 'c', 'opencl', 'fractals', 'newtons-method'],
 ['visual-studio-2013', 'web-deployment-project'],
 ['android'],
 ['logging', 'nlog'],
 ['regex', 'download', 'wget'],
 ['javascript', 'performance', 'highcharts'],
 ['macos', 'bash', 'command-line', 'command-line-arguments'],
 ['mysql', 'cakephp', 'sql-update', 'cakephp-2.0'],
 ['php', 'html', 'css', 'twitter'],
 ['c#', 'azure'],
 ['python', 'notepad++'],
 ['java', 'linux'],
 ['javascript', 'php', 'charts', 'google-visualization'],
 ['linux', 'assembly'],
 ['java', 'object', 'arraylist', 'abstract-class', 'extends'],
 ['java', 'eclipse', 'postgresql', 'jpa', 'persistence'],
 ['colors', 'leaflet'],
 ['java', 'mysql', 'java-ee', 'jdbc'],
 ['css3', 'css-animations'],
 ['c', 'linux', 'ubun